# Extract Poses into NPY Files (Mimic PoseRAC's pre_test.py)

In [1]:
import numpy as np
import pandas as pd
import glob
import os
import sys
import deeplabcut

2024-09-04 22:28:41.351592: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 22:28:43.078210: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:
2024-09-04 22:28:43.078312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:
2024-09-04 22:28:43.078319: W tensorflow/compiler/tf2tensorrt/utils/py_uti

Loading DLC 2.3.10...


In [2]:
# System alterations

# Set the print options to avoid truncation
np.set_printoptions(threshold=sys.maxsize) # Default threshold is 1000

In [3]:
# Note that parameters of this project can be seen at: *openfield-Pranav-2018-10-30/config.yaml*
PATH_CONFIG_FILE = os.path.join(os.getcwd(), 'examples', 'CowBytes-Single-Sadat-2024-06-30', 'config.yaml')
deeplabcut.load_demo_data(PATH_CONFIG_FILE)

This is not an official demo dataset.
Loaded, now creating training data...
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [4]:
# Define test poses destination directory and video file paths [EDIT HERE]
TEST_POSES_DEST_DIR = os.path.join(os.getcwd(), 'examples', 'BiteCountA_pose', 'test_poses')
ROOT_VIDEO_DIR = os.path.join(os.getcwd(), 'examples', 'BiteCountA_pose', 'video')
TRAIN_VIDEO_DIR = os.path.join(ROOT_VIDEO_DIR, 'train')
VALID_VIDEO_DIR = os.path.join(ROOT_VIDEO_DIR, 'valid')
TEST_VIDEO_DIR = os.path.join(ROOT_VIDEO_DIR, 'test')

In [5]:
# Create test_poses directory
os.makedirs(TEST_POSES_DEST_DIR, exist_ok=True)

In [ ]:
# Extract poses into CSV files
video_dir = [TRAIN_VIDEO_DIR, VALID_VIDEO_DIR, TEST_VIDEO_DIR]

for dir in video_dir:
    deeplabcut.analyze_videos(PATH_CONFIG_FILE,dir,save_as_csv=True)

In [7]:
# Load all generated .csv files
csv_files = glob.glob(os.path.join(ROOT_VIDEO_DIR, '**', '*.csv'), recursive=True)
print("Number of generated CSV Files:", len(csv_files))
print(csv_files[:5])

Number of generated CSV Files: 90
['/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/video/valid/056837a2b71e7d93ad65e388dbb0c670a0a440c3ba4f88d82d728ff49d77844c_1DLC_resnet50_CowBytes-SingleJun30shuffle1_288000.csv', '/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/video/valid/117f0eeafe8d5ac58b984131dff27ba3e285203d3e3ea7d090f300f1e33b3c4b_1DLC_resnet50_CowBytes-SingleJun30shuffle1_288000.csv', '/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/video/valid/73c745847dc41f6e0c54fefcadfd13613a200d58b7073017124483409638c9be_1DLC_resnet50_CowBytes-SingleJun30shuffle1_288000.csv', '/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/video/valid/9c08225dd12cdfdfbdc00043de3d390c275458ebe07d07114df2fedb056f4e08_1DLC_resnet50_CowBytes-SingleJun30shuffle1_288000.csv', '/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/video/valid/95040dc15ce9ada82353103a13d1b395c937a9ea6747159f631bfa492b983

In [33]:
def normalise(datapoints, dimensions):
    x_max = np.expand_dims(np.max(datapoints[:, 0::3], axis=1), 1)
    x_min = np.expand_dims(np.min(datapoints[:, 0::3], axis=1), 1)

    y_max = np.expand_dims(np.max(datapoints[:, 1::3], axis=1), 1)
    y_min = np.expand_dims(np.min(datapoints[:, 1::3], axis=1), 1)

    datapoints[:, 0::3] = (datapoints[:, 0::3] - x_min) / (x_max - x_min)
    datapoints[:, 1::3] = (datapoints[:, 1::3] - y_min) / (y_max - y_min)

    if dimensions == 3:
        z_max = np.expand_dims(np.max(datapoints[:, 2::3], axis=1), 1)
        z_min = np.expand_dims(np.max(datapoints[:, 2::3], axis=1), 1)
        datapoints[:, 2::3] = (datapoints[:, 2::3] - z_min) / (z_max - z_min)
    else:
        datapoints[:, 2::3] = 0
    return datapoints

In [ ]:
for csv_file in csv_files:
    # Process CSV file contents
    df = pd.read_csv(csv_file, header=None)
    df = df.drop([0, 1, 2]).reset_index(drop=True)
    df.drop(columns=df.columns[0], axis=1, inplace=True)
    datapoints = df.values.astype(np.float32)
    datapoints[:, 2::3] = 0
    datapoints = normalise(datapoints, 2)

    # Save NPY file with datapoints
    dest_file = os.path.join(TEST_POSES_DEST_DIR, os.path.basename(csv_file.split('DLC')[0]))
    np.save(dest_file, datapoints)
    print("Created file: ", dest_file)

In [ ]:
# Remove redundant files in video directories
files = glob.glob(os.path.join(ROOT_VIDEO_DIR, '**', '*.*'), recursive=True)

# Delete non-JPG files
for file in files:
    if not file.endswith('.mp4'):
        try:
            os.remove(file)
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")